In [ ]:
!git clone https://github.com/suppukerr/neo4j_project/

fatal: destination path 'neo4j_project' already exists and is not an empty directory.


In [ ]:
import pickle
with open('/content/neo4j_project/pnin.pickle', 'rb') as handle:
  book = pickle.load(handle)
book[0] #первое предложение документа

[['in', 'april'],
 ['april', 'a'],
 ['a', 'brazilian'],
 ['brazilian', 'human'],
 ['human', 'right'],
 ['right', 'group'],
 ['group', 'torture'],
 ['torture', 'never'],
 ['never', 'again'],
 ['again', 'awarded'],
 ['awarded', 'the'],
 ['the', 'five'],
 ['five', 'it'],
 ['it', 'chico'],
 ['chico', 'mendes'],
 ['mendes', 'medal'],
 ['medal', 'because'],
 ['because', 'their'],
 ['their', 'right'],
 ['right', 'had'],
 ['had', 'been'],
 ['been', 'violated']]

In [ ]:
!pip install py2neo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tqdm import tqdm

In [ ]:
from py2neo import Graph, Node, Relationship
import string

In [ ]:
url = 'bolt://44.202.237.157:7687'

pwd = 'radiuses-detents-seaman'

In [ ]:
graphnabok = Graph(url, user="neo4j", password=pwd)

In [ ]:
import re
import pickle
import nltk
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
patterns = "[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\-]+"
def lemmatize(doc):
    tokens = []
    doc = re.sub(patterns, ' ', doc)
    for i in range(len(word_tokenize(doc)) - 1):
        token = word_tokenize(doc)[i].strip()
        word = token.lower()
        token = lemmatizer.lemmatize(token)
        next_token = word_tokenize(doc)[i+1].strip()
        next_word = next_token.lower()
        next_token = lemmatizer.lemmatize(next_token)
        tokens.append([token, next_token])
    if len(tokens) > 2:
        return tokens
    return None

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


добавляем предложения в корпус

In [ ]:
def add_sent(punk):
  tx = graphnabok.begin()
  a = Node("Word", word=punk[0])
  tx.create(a)
  b = Node("Word", word=punk[1])
  ab = Relationship(a, "NEXT_WORD", b)
  tx.create(ab)
  graphnabok.commit(tx)

альтернативный способ добавлять предложения в корпус

In [ ]:
# альтернативный способ добавлять предложения в корпус
def add_sentence(senten):
  tx = graphnabok.begin()
  tx.run(
          f'''FOREACH (t IN {senten} | 
          MERGE (w0:Word {{word: t[0]}})
          MERGE (w1:Word {{word: t[1]}})
          CREATE (w0)-[:NEXT_WORD]->(w1)
          )'''
      )
  graphnabok.commit(tx)

In [ ]:
#загрузка базы
for punk in tqdm(book):
  if punk is not None:
    add_sentence(punk)

100%|██████████| 1001/1001 [17:21<00:00,  1.04s/it]


In [ ]:
query = 'MATCH (n) RETURN COUNT(n)'
result = graphnabok.query(query)
print(result.to_data_frame())

   COUNT(n)
0      6721


удаление точки, отношения

In [ ]:
# id = 20 
id_20 = graphnabok.evaluate("MATCH (n) where id(n) = 20 RETURN n") 
# удалить точку
graphnabok.delete(id_20)

In [ ]:
graphnabok.exists(id_20)

True

загрузка данных

In [ ]:
# for punk in tqdm(sents):
#   if punk is not None:
#     add_sentence(lemmatize(punk))

100%|██████████| 6/6 [00:00<00:00, 16.47it/s]


берём все слова слева

In [ ]:
left_query = '''
    MATCH (s:Word {word: $word})
    MATCH (w:Word)-[:NEXT_WORD]->(s)
    RETURN w.word as word
'''

In [ ]:
def left_context(theword):
  ls = set()
  tx = graphnabok.begin()
  l = tx.run(left_query, word=theword)
  for t in l:
     ls.add(t['word'])
  return ls

In [ ]:
left_context('go')

{'and', 'ban', 'pokemon', 'the', 'to', 'would'}

In [ ]:
left_context('president')

{'college',
 'critiqued',
 'former',
 'general',
 'korean',
 'new',
 'nra',
 'of',
 'removing',
 'romanian',
 's',
 'the',
 'vice'}

берём все слова справа

In [ ]:
right_query = '''
    MATCH (s:Word {word: $word})
    MATCH (w:Word)<-[:NEXT_WORD]-(s)
    RETURN w.word as word
'''

In [ ]:
def right_context(theword):
  rs = set()
  tx = graphnabok.begin()
  r = tx.run(right_query, word=theword)
  for s in r:
     rs.add(s['word'])
  return rs

In [ ]:
right_context('go')

{'back', 'on', 'out', 'record', 'to', 'too', 'with'}

In [ ]:
right_context('think')

word,next_word,next_next_word
think,we,ll
think,we,were
think,we,typically


In [ ]:
def jaccard_similarity(w1, w2):
    nominator = w1.intersection(w2)
    denominator = w1.union(w2)
    similarity = len(nominator)/len(denominator)
    return similarity

In [ ]:
def paradigSimilarity(w1, w2):
    return (jaccard_similarity(left_context(w1), left_context(w2)) + 
            jaccard_similarity(right_context(w1), right_context(w2))) / 2.0

In [ ]:
paradigSimilarity('new', 'old')

0.038461538461538464

In [ ]:
paradigSimilarity('school', 'university')

0.016666666666666666

In [ ]:
left_query_next = '''
    MATCH (s:Word {word: $word})
    MATCH (w:Word)-[:NEXT_WORD]->(s)
    MATCH (d:Word)-[:NEXT_WORD]->(w)
    RETURN d.word as prev_prev_word, w.word as prev_word, s.word as word
'''

In [ ]:
def left_context_next(theword):
  tx = graphnabok.begin()
  l = tx.run(left_query_next, word=theword)
  return l

In [ ]:
left_context_next('there')

prev_prev_word,prev_word,word
exceeded,that,there
accusation,that,there
provided,that,there


In [ ]:
right_query_next = '''
    MATCH (s:Word {word: $word})
    MATCH (w:Word)<-[:NEXT_WORD]-(s)
    MATCH (d:Word)<-[:NEXT_WORD]-(w)
    RETURN s.word as word, w.word as next_word, d.word as next_next_word
'''

In [ ]:
def right_context(theword):
  tx = graphnabok.begin()
  r = tx.run(right_query, word=theword)
  return r

In [ ]:
right_context('president')

word,next_word,next_next_word
president,is,a
president,is,a
president,is,small


In [ ]:
both_query = '''MATCH (s:Word)
      MATCH (w:Word)-[:NEXT_WORD]->(s)
      WITH collect(DISTINCT w.word) as left1, s
      RETURN left1, s
      '''
tx = graphnabok.begin()
both = tx.run(both_query)
both

left1,s
"['in', 'club']",(_330:Word {word: 'forming'})
"['in', 'both', 'east', 'ghosh']",(_1178:Word {word: 'london'})
"['in', 'and', 'of']",(_4781:Word {word: 'certain'})


Наработки

In [ ]:
sim_query = '''
MATCH (s:Word)
// правый и левый контексты, собираем их
MATCH (w:Word)-[:NEXT_WORD]->(s)
WITH collect(DISTINCT w.word) as left1, s
MATCH (w:Word)<-[:NEXT_WORD]-(s)
WITH left1, s, collect(DISTINCT w.word) as right1
// собираем все остальные слова, отличные от центрального -- искомого слова
MATCH (o:Word) WHERE NOT s = o
WITH left1, right1, s, o
// берём следующие правые и следующие левые
MATCH (w:Word)-[:NEXT_WORD]->(o)
WITH collect(DISTINCT w.word) as left1_o, s, o, right1, left1
MATCH (w:Word)<-[:NEXT_WORD]-(o)
WITH left1_o, s, o, right1, left1, collect(DISTINCT w.word) as right1_o
// считаем юнион интесект справа
WITH FILTER(x IN right1 WHERE x IN right1_o) as r1_intersect,
  (right1 + right1_o) AS r1_union, s, o, right1, left1, right1_o, left1_o
// считаем юнион интесект слева
WITH FILTER(x IN left1 WHERE x IN left1_o) as l1_intersect,
  (left1 + left1_o) AS l1_union, r1_intersect, r1_union, s, o
WITH DISTINCT r1_union as r1_union, l1_union as l1_union, r1_intersect, l1_intersect, s, o
//считаем по формуле джаккарда
WITH 1.0*length(r1_intersect) / length(r1_union) as r1_jaccard,
  1.0*length(l1_intersect) / length(l1_union) as l1_jaccard,
  s, o
WITH s, o, r1_jaccard, l1_jaccard, r1_jaccard + l1_jaccard as sim
//задаём это как отношение и самостоятельное значение
CREATE UNIQUE (s)-[r:RELATED_TO]->(o) SET r.paradig = sim;
'''

tx = graphnabok.begin()
tx.run(sim_query)

выдать наиболее похожие друг на друга по симиларити

In [ ]:
check_query = '''MATCH (s)-[r:RELATED_TO]->(o) 
RETURN s.word,o.word,r.paradig AS sim ORDER BY sim DESC LIMIT 100;
'''
tx = graphnabok.begin()
tx.run(check_query)

In [ ]:
#удалить всё
graphnabok.delete_all()